A3C Algorithm to solve a simple game. Not currently converging to a solution.
Lots of the code was taken from this repo https://github.com/awjuliani/DeepRL-Agents

In [ ]:
!apt-get install -y python-opengl
!pip install gym[atari]
!pip install JSAnimation

In [ ]:
import numpy as np
import random
import itertools
import scipy.misc
import matplotlib.pyplot as plt


class gameOb():
    def __init__(self,coordinates,size,intensity,channel,reward,name):
        self.x = coordinates[0]
        self.y = coordinates[1]
        self.size = size
        self.intensity = intensity
        self.channel = channel
        self.reward = reward
        self.name = name
        
class gameEnv():
    class ActionSpace(object):
        
        def __init__(self, n):
            self.n = n
    def __init__(self,partial,size):
        self.sizeX = size
        self.sizeY = size
        self.actions = 4
        self.action_space = object()
        self.action_space = self.ActionSpace(self.actions)
        self.objects = []
        self.partial = partial
        a = self.reset()
        plt.imshow(a,interpolation="nearest")
        
        
    def reset(self):
        self.objects = []
        hero = gameOb(self.newPosition(),1,1,2,None,'hero')
        self.objects.append(hero)
        bug = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug)
        hole = gameOb(self.newPosition(),1,1,0,-1,'fire')
        self.objects.append(hole)
        bug2 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug2)
        hole2 = gameOb(self.newPosition(),1,1,0,-1,'fire')
        self.objects.append(hole2)
        bug3 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug3)
        bug4 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug4)
        state = self.renderEnv()
        self.state = state
        return state

    def moveChar(self,direction):
        # 0 - up, 1 - down, 2 - left, 3 - right
        hero = self.objects[0]
        heroX = hero.x
        heroY = hero.y
        penalize = -0.05
        if direction == 0 and hero.y >= 1:
            hero.y -= 1
        if direction == 1 and hero.y <= self.sizeY-2:
            hero.y += 1
        if direction == 2 and hero.x >= 1:
            hero.x -= 1
        if direction == 3 and hero.x <= self.sizeX-2:
            hero.x += 1     
        if hero.x == heroX and hero.y == heroY:
            penalize = -0.05
        self.objects[0] = hero
        return penalize
    
    def newPosition(self):
        iterables = [ range(self.sizeX), range(self.sizeY)]
        points = []
        for t in itertools.product(*iterables):
            points.append(t)
        currentPositions = []
        for objectA in self.objects:
            if (objectA.x,objectA.y) not in currentPositions:
                currentPositions.append((objectA.x,objectA.y))
        for pos in currentPositions:
            points.remove(pos)
        location = np.random.choice(range(len(points)),replace=False)
        return points[location]

    def checkGoal(self):
        others = []
        for obj in self.objects:
            if obj.name == 'hero':
                hero = obj
            else:
                others.append(obj)
        ended = False
        for other in others:
            if hero.x == other.x and hero.y == other.y:
                self.objects.remove(other)
                if other.reward == 1:
                    self.objects.append(gameOb(self.newPosition(),1,1,1,1,'goal'))
                else: 
                    self.objects.append(gameOb(self.newPosition(),1,1,0,-1,'fire'))
                return other.reward,False
        if ended == False:
            return 0.0,False

    def renderEnv(self):
        #a = np.zeros([self.sizeY,self.sizeX,3])
        a = np.ones([self.sizeY+2,self.sizeX+2,3])
        a[1:-1,1:-1,:] = 0
        hero = None
        for item in self.objects:
            a[item.y+1:item.y+item.size+1,item.x+1:item.x+item.size+1,item.channel] = item.intensity
            if item.name == 'hero':
                hero = item
        if self.partial == True:
            a = a[hero.y:hero.y+3,hero.x:hero.x+3,:]
        b = scipy.misc.imresize(a[:,:,0],[84,84,1],interp='nearest')
        c = scipy.misc.imresize(a[:,:,1],[84,84,1],interp='nearest')
        d = scipy.misc.imresize(a[:,:,2],[84,84,1],interp='nearest')
        a = np.stack([b,c,d],axis=2)
        return a

    def step(self,action):
        penalty = self.moveChar(action)
        reward,done = self.checkGoal()
        state = self.renderEnv()
        if reward == None:
            print(done)
            print(reward)
            print(penalty)
            return state,(reward+penalty),done, None
        else:
            return state,(reward+penalty),done, None
        
import tensorflow as tf
import tensorflow.contrib.slim as slim
import gym
import random
import multiprocessing
import threading
from collections import namedtuple
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))


def wrapper(func):
    def wrapped(*args, **kwargs):
        if args[0] == 'test':
            env = gameEnv(partial=False, size=9)
        else:
            env = func(*args, **kwargs)

        return env
    return wrapped

gym.make = wrapper(gym.make)
"""
env = gym.make('Assault-v0')
observation = env.reset()
action_size = env.action_space.n
frames = []
for i in range(100):
    observation, reward, done, info = env.step(random.randint(0, 6))
    frames.append(env.render(mode='rgb_array'))
    if done:
        break
display_frames_as_gif(frames)
"""

def update_vars(from_scope, to_scope):
    from_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, from_scope)
    to_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, to_scope)
    ops = []
    for to_var, from_var in zip(to_vars, from_vars):
        op = to_var.assign(from_var)
        ops.append(op)
        
    return ops

class Network(object):

    def __init__(self, sess, input_dims, action_size, hidden_size, scope, trainer):
        self.sess = sess
        self.input_dims = input_dims
        self.action_size = action_size
        self.hidden_size = hidden_size
        self.scope = scope
        self.trainer = trainer
        self.calculator = FeedCalculator()


        with tf.variable_scope(scope):
            self.input = tf.placeholder(
                shape=[None] + list(self.input_dims),
                dtype=tf.float32
            )
            
            self.input /= 255.0
            self.conv1 = slim.conv2d(
                inputs=self.input,
                num_outputs=16,
                kernel_size=[4, 4],
                stride=[2, 2],
                activation_fn=tf.nn.elu,
                padding='VALID'
            )

            self.conv2 = slim.conv2d(
                inputs=self.conv1,
                num_outputs=32,
                kernel_size=[8, 8],
                stride=[4, 4],
                activation_fn=tf.nn.elu,
                padding='VALID'
            )
            
            self.connected = slim.fully_connected(
                tf.layers.flatten(self.conv2),
                self.hidden_size,
                activation_fn=tf.nn.elu
            )

            lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.hidden_size, state_is_tuple=True)
            #self.rnn_zero_state = lstm_cell.zero_state(1, dtype=tf.float32)
            self.rnn_zero_state = (
                np.zeros([1, self.hidden_size], dtype="float32"),
                np.zeros([1, self.hidden_size], dtype="float32")
            )
            
            self.c_in = tf.placeholder(
                shape=[1, self.hidden_size],
                dtype=tf.float32
            )
            
            self.h_in = tf.placeholder(
                shape=[1, self.hidden_size],
                dtype=tf.float32
            )
            
            self.state_in = tf.nn.rnn_cell.LSTMStateTuple(self.c_in, self.h_in)
    
            output, state = tf.nn.dynamic_rnn(
                cell=lstm_cell,
                inputs=tf.expand_dims(self.connected,[0]),
                initial_state=self.state_in,
                time_major=False,
            )

            self.rnn_state = state

            output = tf.reshape(output, shape=[-1, self.hidden_size])
            #output = tf.reshape(self.connected, shape=[-1, self.hidden_size])
            self.policy = slim.fully_connected(
                output,
                self.action_size,
                activation_fn=tf.nn.softmax
            )

            self.value = slim.fully_connected(
                output,
                1
            )

            self.advantage = tf.placeholder(shape=[None], dtype=tf.float32)
            self.action = tf.placeholder(shape=[None], dtype=tf.int32)
            self.target_v = tf.placeholder(shape=[None], dtype=tf.float32)
            self.action_onehot = tf.one_hot(self.action, self.action_size)
            
            responsible_outputs = tf.reduce_sum(
                self.policy * self.action_onehot,
                [1]
            )
            
            self.policy_loss = - tf.reduce_sum(
                tf.log(responsible_outputs) * self.advantage,
            )
            
            self.value_loss = tf.reduce_sum(
                tf.square(self.value - self.target_v),
            )
            self.entropy_loss = - tf.reduce_sum(
                tf.log(self.policy) * self.policy,
            )

            self.loss = self.policy_loss + .25 * self.value_loss - .01 * self.entropy_loss
            local_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, self.scope)
            grads = tf.gradients(self.loss, local_vars)
            grads, norms = tf.clip_by_global_norm(grads, 40.0)
            global_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'global')
            self.minimize = self.trainer.apply_gradients(zip(grads, global_vars))
            self.update_local_ops = update_vars("global", self.scope)

    def runUpdateVariables(self):

        return self.sess.run(self.update_local_ops)

    def runPolicyValueState(self, inputs, initial_rnn_state):
        
        action_dist, value, state = sess.run(
            [self.policy, self.value, self.rnn_state],
            feed_dict={
                self.input: inputs,
                self.c_in: initial_rnn_state[0],
                self.h_in: initial_rnn_state[1]
            }
        )
        
        return action_dist, value, state
        
    def runMinimize(self, experiences, initial_rnn_state, next_value):
        
        advantages, target_values = \
            self.calculator.advantages_and_target_values(
                experiences, next_value
            )

        inputs = np.array([e.state for e in experiences])
        actions = np.array([e.action for e in experiences])
        
        _ = sess.run(
            [self.minimize],
            feed_dict={
                self.input: inputs,
                self.c_in: initial_rnn_state[0],
                self.h_in: initial_rnn_state[1],
                self.advantage: advantages,
                self.target_v: target_values,
                self.action: actions
            }
        )

import scipy.signal

class FeedCalculator(object):
    
    def __init__(self, gamma=.99):
        
        self.gamma = gamma
    
    def discount(self, array):
        
        return scipy.signal.lfilter(
            [1],
            [1, -self.gamma],
            array[: : -1],
            axis=0
        )[: : -1]

    def advantages_and_target_values(self, experiences, next_value):
        
        rewards = np.array([e.reward for e in experiences] + [next_value])
        values = np.array([e.value for e in experiences] + [next_value])
        target_values = self.discount(rewards)[:-1]
        advantages = rewards[:-1] + self.gamma * values[1: ] - values[: -1]
        return advantages, target_values
        
class BufferFullException(Exception):
    pass

Experience = namedtuple(
    "Experience",
    "state action reward next_state done value"
)


class ExperienceBuffer(object):
    
    def __init__(self, size):
        
        self._buffer = []
        self._size = size

    @property
    def is_full(self):
        
        return len(self._buffer) == self._size

    @property
    def experiences(self):
        
        return self._buffer

    def reset(self):
        
        self._buffer = []

    def add(self, experience):
        
        if len(self._buffer) >= self._size:
            raise BufferFullException(
                "Buffer reached max size of {}".format(self._size)
            )
            
        self._buffer.append(experience)
        
class Worker(threading.Thread):
    
    MAX_EPISODE_LENGTH = 300
    BUFFER_SIZE = 50
    def __init__(self, name, env_name, sess, trainer, hidden_size, global_network):
        
        super(Worker, self).__init__()
        self.name = "worker_" + str(name)
        self.env = gym.make(env_name)
        self.global_network = global_network
        self.network = Network(
            scope=self.name,
            sess=sess,
            input_dims=self.env.reset().shape,
            action_size=self.env.action_space.n,
            trainer=trainer,
            hidden_size=hidden_size
        )
        self.buffer = ExperienceBuffer(self.BUFFER_SIZE)
        self.initial_rnn_state = None
        self.episode_count = 0
        self.episode_rewards = []
        self.running_rewards = 0

    def print_rewards(self):
        print("worker: {} episode: {} reward: {}".format(
            self.name,
            self.episode_count,
            np.mean(self.episode_rewards[-25:])
        ))

    def reset_buffer(self):

        self.buffer.reset()

    def run(self):
        self.work()

    def work(self):
        
        for j in range(5000):
            self.episode_count = j
            if self.episode_count % 1 == 0:
                self.network.runUpdateVariables()
            if j != 0:
                self.episode_rewards.append(self.running_rewards)
                self.running_rewards = 0
            if self.episode_count % 25 == 0:
                self.print_rewards()

            state = self.env.reset()
            self.initial_rnn_state = self.network.rnn_zero_state
            self.running_rewards = 0
            for i in range(self.MAX_EPISODE_LENGTH):
                policy, value, rnn_state = self.network.runPolicyValueState(
                    inputs=[state],
                    initial_rnn_state = self.initial_rnn_state
                )
                policy = policy[0]
                value = value[0][0]
                action = np.random.choice(policy, p=policy)
                action = np.argmax(action == policy)
                state1, reward, done, _ = self.env.step(action)
                self.running_rewards += reward
                experience = Experience(
                    state,
                    action,
                    reward,
                    state1,
                    done,
                    value
                )

                self.buffer.add(experience)

                if done == True:
                    self.network.runMinimize(
                        self.buffer.experiences,
                        self.initial_rnn_state,
                        next_value=0.0
                    )
                    self.reset_buffer()
                    break
                if self.buffer.is_full:
                    
                    _, next_value, _ = self.network.runPolicyValueState(
                        inputs=[state1],
                        initial_rnn_state=rnn_state
                    )
                    self.network.runMinimize(
                        self.buffer.experiences,
                        self.initial_rnn_state,
                        next_value
                    )
                    self.reset_buffer()
                    self.initial_rnn_state = rnn_state

hidden_size = 256
trainer = tf.train.AdamOptimizer(learning_rate=.0001)
n_workers = multiprocessing.cpu_count()
workers = []
worker_threads = []
#env_name = "Pong-v0"
env_name = "test"
#env_name = "Assault-v0"
env = gym.make(env_name)

tf.reset_default_graph()

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    global_network = Network(
        scope="global",
        sess=sess,
        input_dims=env.reset().shape,
        action_size=env.action_space.n,
        trainer=trainer,
        hidden_size=hidden_size
    )


    for i in range(n_workers):
        w = Worker(
            name=i,
            env_name=env_name,
            sess=sess,
            trainer=trainer,
            hidden_size=hidden_size,
            global_network=global_network
        )


        workers.append(w)

    sess.run(tf.global_variables_initializer())
    for w in workers:
        w.start()
        
    coord.join(workers)




